In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from keras.datasets import mnist

import visualkeras

In [2]:
# loading dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X = np.concatenate([X_train, X_test], axis = 0)
y = np.concatenate([y_train, y_test], axis = 0)

# reshapring 
X = X.reshape(X.shape[0], 28, 28, 1) 

# casting to float 
X.astype('float32')

# normalizing 
X = X / 255

# encoding output values
y = to_categorical(y)

In [3]:
INPUT_SHAPE = (28,28,1)
OUTPUT_SHAPE = 10
BATCH_SIZE = 6
EPOCHS = 2 
VERBOSE = 2
K_FOLDS = 10

In [4]:
def create_model():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=INPUT_SHAPE))
    model.add(MaxPool2D((2,2)))

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPool2D((2,2)))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        optimizer='adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    
    return model

In [5]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [6]:
scores = []

for i in range(K_FOLDS):
    print("Training on Fold: ",i+1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3
                                               ,random_state = np.random.randint(1,1000, 1)[0])
    
    model = create_model()
    model.fit(
        X_train, 
        y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=VERBOSE,
        validation_split=0.3
    )
    
    _, score = model.evaluate(X_test, y_test, verbose = 0)
    scores.append(score)
   
    print("======="*12, end="\n\n\n")

Training on Fold:  1
Epoch 1/2
5717/5717 - 42s - loss: 0.2063 - accuracy: 0.9386 - val_loss: 0.0709 - val_accuracy: 0.9787 - 42s/epoch - 7ms/step
Epoch 2/2
5717/5717 - 36s - loss: 0.0776 - accuracy: 0.9776 - val_loss: 0.0568 - val_accuracy: 0.9843 - 36s/epoch - 6ms/step


Training on Fold:  2
Epoch 1/2
5717/5717 - 38s - loss: 0.1965 - accuracy: 0.9401 - val_loss: 0.0661 - val_accuracy: 0.9813 - 38s/epoch - 7ms/step
Epoch 2/2
5717/5717 - 46s - loss: 0.0751 - accuracy: 0.9792 - val_loss: 0.0624 - val_accuracy: 0.9825 - 46s/epoch - 8ms/step


Training on Fold:  3
Epoch 1/2
5717/5717 - 51s - loss: 0.1971 - accuracy: 0.9413 - val_loss: 0.0796 - val_accuracy: 0.9763 - 51s/epoch - 9ms/step
Epoch 2/2
5717/5717 - 45s - loss: 0.0777 - accuracy: 0.9786 - val_loss: 0.0669 - val_accuracy: 0.9813 - 45s/epoch - 8ms/step


Training on Fold:  4
Epoch 1/2
5717/5717 - 39s - loss: 0.2085 - accuracy: 0.9353 - val_loss: 0.0710 - val_accuracy: 0.9790 - 39s/epoch - 7ms/step
Epoch 2/2
5717/5717 - 44s - loss: 0

In [9]:

print(f"K{K_FOLDS} Fold cross results:")
print("Accuracy:", np.mean(scores) * 100)
print("Std:",      np.std(scores) * 100)
print("K value:", len(scores))

K10 Fold cross results:
Accuracy: 98.28523755073547
Std: 0.15758266165534981
K value: 10
